In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import re


In [12]:
#First is HBS

source='https://www.hbs.edu/coursecatalog/indexcourse.html'

html = urlopen(str(source))
soup = BeautifulSoup(html, 'html.parser')

tdlist = soup.find_all('td')

reflist = []
for i in range(len(tdlist)):
    if str(tdlist[i])[:27] == '<td><a href="/coursecatalog':
        reflist.append(str(tdlist[i]))
    
#This code is necessary because HBS has course descriptions on different pages, rather than collected in one URL.

linklist = []
for i in range(len(reflist)):
    linklist.append(reflist[i][14:37])
    
#Creating a list of link extensions    
    
descraw = []

for i in range(len(linklist)):
    link = urlopen('https://www.hbs.edu/'+ linklist[i])
    descsoup = BeautifulSoup(link, 'html.parser')
    descraw.append(descsoup.findAll('p'))
    
#navigating to link extensions and retrieving course descriptions    


titlelist = []
for i in range(len(reflist)):
    titlelist.append(re.sub('<[^<]+?>', '', str(reflist[i]))) 
#titlelist


hbsdf = pd.DataFrame(titlelist, columns=["Course"])
hbsdf['Description'] = descraw

hbsdf['Source'] = source

for i in range(len(hbsdf)):
    if i >0:
        hbsdf['Source'][i]= ''
    else:
        continue

#This writes everything to a dataframe. Easy to export to excel. The original data is stored in a list, so it could
#easily be written to a different kind of object, but I have used dataframes here for convenience.
    

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [14]:
#Next is Wharton. This was the first code that I wrote, so it is a bit clunkier. Also, this includes prerequisites.
#I have decided to remove this from the final version, because it is outside the scope of the original project.

source='https://mgmt.wharton.upenn.edu/programs/mba/course-descriptions/'

html = urlopen(str(source))
soup = BeautifulSoup(html, 'html.parser')

full_text = soup.get_text()

title = soup.select('h3')
para = soup.select('p')

for i in range(len(title)):
    title[i] = re.sub('<[^<]+?>', '', str(title[i]))

#you can see here I am scrubbing everything unnecessarily early in the process. 

title1df = pd.DataFrame(title, columns=['Course'])

title1df['Source'] = source

#The above two sections are taking the raw course catalog and putting it into a dataframe. The below sections are removing the
#'prerequisites' columns. Because this comes from the original code, where I created an excel file with only course names, 
#there is some redundancy here. Keeping it in case the code comes in handy later. 

for i in range(len(title1df)):
    if i >0:
        title1df['Source'][i]= ''
    else:
        continue
        
prereq = []

for strong_tag in soup.find_all('strong'):
    prereq.append((strong_tag.text, strong_tag.next_sibling))
    
poplist = []
prereqlist = []

for i in range(len(para)):
    if str(para[i])[:11] == '<p><strong>':
        poplist.append(i)
    else:
        continue

newpara = []

for i in range(len(para)):
    if i in poplist:
        prereqlist.append(para[i])
    else:
        newpara.append(para[i])

        
#All of this is to fix the problem of classes without prerequisites. This is redundant, but the code is included here in 
#case it is useful for another school's formatting (all these websites have different formatting)

fulldf = pd.DataFrame(title[:-3], columns= ['Course'])

fulldf['Source'] = source

for i in range(len(fulldf)):
    if i >0:
        fulldf['Source'][i]= ''
    else:
        continue

fulldf['Description'] = ''

for i in range(len(newpara)):
    fulldf['Description'][i]= re.sub('<[^<]+?>', '', str(newpara[i]))

newpara
fulldf['Prerequisites'] = ""       

for i in range(len(fulldf)):
    for j in range(len(poplist)):
        if i == poplist[j]:
            fulldf['Prerequisites'][i]= re.sub('<[^<]+?>', '', str(prereqlist[j]))
        else: continue   
fullcols = fulldf.columns.tolist()

newcols = ['Course','Description', 'Prerequisites', 'Source']

whartondf = fulldf[newcols]


In [17]:
#Stanford had problably the simplest, best-formatted website. Bless the people at Stanford who are running this website. 

source = 'https://exploredegrees.stanford.edu/graduateschoolofbusiness/#courseinventory'

html = urlopen(str(source))
soup = BeautifulSoup(html, 'html.parser')

#soup.find_all()
title = soup.select('strong')

for i in range(len(title)):
    title[i] = re.sub('<[^<]+?>', '', str(title[i]))
    
titlelist = []
desclist = []

for i in soup.findAll('p'):
    if str(i)[:28] == '<p class="courseblocktitle">':
        titlelist.append(i)
    
    elif str(i)[:27] == '<p class="courseblockdesc">':
        desclist.append(i)
        
        
#Below is simply to remove html tags from the text.
        
for i in range(len(title)):
    title[i] = re.sub('<[^<]+?>', '', str(title[i]))     
    
    

stanforddf = pd.DataFrame(titlelist, columns=["Course"])

stanforddf['Description'] = desclist

for i in range(len(stanforddf)):
    stanforddf['Course'][i] = re.sub('<[^<]+?>', '', str( stanforddf['Course'][i]))
    stanforddf['Description'][i] = re.sub('<[^<]+?>', '', str( stanforddf['Description'][i]))
    


stanforddf['Source'] = source

for i in range(len(stanforddf)):
    if i >0:
        stanforddf['Source'][i]= ''
    else:
        continue
    
    


In [13]:
#This will have to wait until I have a more finished product to test on. 

#import pdfkit

#pdfkit.from_string(hbsdf, 'out.pdf')

In [ ]:
#In class, making notes for later. Can use .strip to remove whitespace. 

#look for class or id when you are doing .find_all as an argument within the function.